# Notebook to generate offxml with specific smiles patterns

The notebook reads in a mol2 file and a frcmod file and gives back an openforcefield offxml file. It only modifies the LJ parameters. All other parameters are taken from the smirnoff offxml file specified. 

I read in all valence terms from smirnoff99Frosst and write them to our new force field file. I also included the header for the nonbondes section of the smirnoff force field format. 

In [1]:
offxml_file = open('/home/mschauperl/projects/lj_sam/smirnoff99FrosstHRES.offxml')
offxml_lines = offxml_file.readlines()
offxml_file.close()
offxml_file = open('/home/mschauperl/projects/lj_sam/SDLJ.offxml','w')
for line in offxml_lines:
    if 'NonbondedForce' in line:
        offxml_file.write(line)
        break
    else:
        offxml_file.write(line)

I read in the atomtypes in the order they appear in the mol2 file. Unfortunately, openeye converts the atomtypes always to Sybyl atom types. I am not sure if there is a way to avoid this. This is therefore a workaround this problem. 

In [2]:
# Important!!!! Order of atom has to be the same in both mol2 files!!!!!!
mol2_filename ="/home/mschauperl/projects/lj_sam/THF.mol2"
mol2sybyl_filename ="/home/mschauperl/projects/lj_sam/THF_sybyl.mol2"

mol2_file = open(mol2_filename,'r')
mol2_lines = mol2_file.readlines()
mol2_file.close()
atomtypes={} # Stores the atomtypes dict with atom index as key (starts with 1)
for i,line in enumerate(mol2_lines):
    if line.startswith('MOL'):
        n_atom = int(mol2_lines[i+1].split()[0])
    if line.startswith("@<TRIPOS>ATOM"):
        for j in range(1,n_atom+1,1):
            atomtypes[j]=(mol2_lines[i+j].split()[5])

In [3]:
from chemper.mol_toolkits import mol_toolkit
from chemper.graphs.cluster_graph import ClusterGraph
from chemper.chemper_utils import create_tuples_for_clusters
from openeye import oechem
#mol = mol_toolkit.MolFromSmiles('CCCO')
mol = mol_toolkit.mols_from_mol2(mol2sybyl_filename)[0]
ifs = oechem.oemolistream(mol2sybyl_filename)
oe_mol = oechem.OEGraphMol()
oechem.OEReadMol2File(ifs, oe_mol)
dir(oe_mol)
smiles_code={}
for num,atom,oe_atom in zip(range(1,n_atom+1),mol.get_atoms(),oe_mol.GetAtoms()):
    index=(atom.get_index())
    graph = ClusterGraph([mol], [[(index,)]], layers='all')
    print(graph.as_smirks())
    smiles_code[num]=graph.as_smirks()
    #print(oe_atom.GetType())


[#6H2X4x2r5+0A:1](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])(-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])-;!@[#1H0X1x0!r+0A])-;@[#8H0X2x2r5+0A]
[#6H2X4x2r5+0A:1](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])(-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;@[#8H0X2x2r5+0A]-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])-;!@[#1H0X1x0!r+0A])-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])-;!@[#1H0X1x0!r+0A]
[#6H2X4x2r5+0A:1](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])(-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;@[#8H0X2x2r5+0A])-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])-;!@[#1H0X1x0!r+0A]
[#1H0X1x0!r+0A:1]-;!@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;@[#8H0X2x2r5+0A]-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])-;!@[#1H0X1x0!r+0A])-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!

RDKit WARNING: [15:26:16] Enabling RDKit 2019.09.1 jupyter extensions


In [4]:
frcmod_file = open('/home/mschauperl/projects/lj_sam/THF.frcmod','r')
frcmod_lines = frcmod_file.readlines() 
frcmod_file.close()
lj_params={}
nb=False
for line in frcmod_lines:
    if line.startswith('NONBON'):
        nb=True
    elif nb ==True:
        entry=line.split()
        lj_params[entry[0]]={'eps':float(entry[1]),'rmin_2':float(entry[2])}

In [5]:
atomtypes

{1: 'bj',
 2: 'bk',
 3: 'bk',
 4: 'bl',
 5: 'bl',
 6: 'bj',
 7: 'bl',
 8: 'bl',
 9: 'bm',
 10: 'bn',
 11: 'bn',
 12: 'bn',
 13: 'bn'}

In [6]:
lj_params

{'bj': {'eps': 1.91156266387, 'rmin_2': 0.08149111612961},
 'bk': {'eps': 1.965148247978, 'rmin_2': 0.07710076675759},
 'bl': {'eps': 1.413258426966, 'rmin_2': 0.02581745462997},
 'bm': {'eps': 1.468801824212, 'rmin_2': 0.1522620202606},
 'bn': {'eps': 1.42197327852, 'rmin_2': 0.02562615167227}}

In [7]:
smiles_code

{1: '[#6H2X4x2r5+0A:1](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])(-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])-;!@[#1H0X1x0!r+0A])-;@[#8H0X2x2r5+0A]',
 2: '[#6H2X4x2r5+0A:1](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])(-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;@[#8H0X2x2r5+0A]-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])-;!@[#1H0X1x0!r+0A])-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])-;!@[#1H0X1x0!r+0A]',
 3: '[#6H2X4x2r5+0A:1](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])(-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;@[#8H0X2x2r5+0A])-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])-;!@[#1H0X1x0!r+0A]',
 4: '[#1H0X1x0!r+0A:1]-;!@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;@[#8H0X2x2r5+0A]-;@[#6H2X4x2r5+0A](-;!@[#1H0X1x0!r+0A])-;!@[#1H0X1x0!r+0A])-;@[#6H

In [8]:
for key, atomtype in atomtypes.items():
    line='<Atom smirks="{}" epsilon="{}" id="n{}" rmin_half="{}"/>\n'.format(smiles_code[key],lj_params[atomtype]['eps'],key,lj_params[atomtype]['rmin_2'])
    offxml_file.write(line)

In [9]:
offxml_file.write('</NonbondedForce>\n')
offxml_file.write('</SMIRNOFF>\n')

12

In [10]:
offxml_file.close()